In [ ]:
# ======================================================================================================================
# BMW SALES TREND FORECASTING & ALERT SYSTEM - CONSOLIDATED NOTEBOOK/SCRIPT
# ======================================================================================================================

import os
import sys
import glob
import logging
import warnings
import webbrowser
import requests
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from pathlib import Path
from datetime import datetime
from plotly.subplots import make_subplots
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Setup for Jupyter Notebook to display plots inline
%matplotlib inline
warnings.filterwarnings('ignore')

# ==================================================================================================
# 1. CONFIGURATION (Adapted from config.py)
# ==================================================================================================

# Determine Project Root (uses current working directory in Notebooks)
try:
    PROJECT_ROOT = Path(__file__).resolve().parent
except NameError:
    PROJECT_ROOT = Path.cwd()

# Output directory
OUTPUT_DIR = PROJECT_ROOT / 'outputs'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def out_path(name: str) -> str:
    """Return a path inside the outputs directory as a string."""
    return str(OUTPUT_DIR / name)

# Plotting Styles
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Pandas Options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Data Sources
DATA_CSV_URL = 'https://raw.githubusercontent.com/StephenEastham/bmw-sales-forecast/refs/heads/main/v251125/BMW-sales-data-2010-2024.csv'
HOWTO_URL = 'https://raw.githubusercontent.com/StephenEastham/bmw-sales-forecast/refs/heads/main/how-to-test.md'
DATA_CSV_FILE = 'BMW-sales-data-2010-2024.csv'
HOWTO_FILE = 'how-to-test.md'

# Parameters
ARIMA_ORDER = (1, 1, 1)
FORECAST_STEPS = 3
TRAIN_TEST_SPLIT = 0.8

# Thresholds
OVERALL_THRESHOLD_MULTIPLIER = 0.8
MODEL_THRESHOLD_MULTIPLIER = 0.8
REGION_THRESHOLD_MULTIPLIER = 0.8
DECLINE_THRESHOLD = 0.15

# Test Mode Settings
TEST_MODE = False
TEST_OVERALL_FORECAST_LOW = True
TEST_MODEL_UNDERPERFORMANCE = True
TEST_REGION_DECLINE = True
TEST_DECLINING_TREND = True

print(f"✅ Configuration loaded. Outputs: {OUTPUT_DIR}")

# ==================================================================================================
# 2. UTILITIES (Adapted from utils.py)
# ==================================================================================================

def setup_logger(log_file='sales_alerts.log'):
    """Setup logging to file and console"""
    logger = logging.getLogger('BMW_Forecast_Logger')
    logger.setLevel(logging.INFO)

    if not logger.handlers:
        fh = logging.FileHandler(out_path(log_file))
        fh.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
        logger.addHandler(fh)

        sh = logging.StreamHandler()
        sh.setFormatter(logging.Formatter('%(levelname)s - %(message)s'))
        logger.addHandler(sh)

    return logger


def print_section(title):
    """Print a formatted section header"""
    print("\n" + "="*80)
    print(title)
    print("="*80)

# ==================================================================================================
# 3. DATA PROCESSING (Adapted from data.py)
# ==================================================================================================

def download_data_file(file_name, data_url):
    """Download data file from URL if not exists"""
    if not os.path.exists(file_name):
        try:
            print(f"Attempting to download {file_name} from {data_url}...")
            response = requests.get(data_url)
            response.raise_for_status()
            with open(file_name, 'wb') as f:
                f.write(response.content)
            print(f"✅ {file_name} downloaded successfully!")
        except requests.exceptions.RequestException as e:
            print(f"❌ Failed to download {file_name}. Error: {e}")
    else:
        print(f"✅ {file_name} already exists.")


def download_required_files():
    download_data_file(DATA_CSV_FILE, DATA_CSV_URL)
    download_data_file(HOWTO_FILE, HOWTO_URL)


def load_and_explore_data(csv_path):
    print_section("📊 DATASET OVERVIEW")
    df = pd.read_csv(csv_path)
    print(f"\n✅ Data loaded successfully! Shape: {df.shape}")
    print(f"\nFirst few rows:\n{df.head(5)}")
    print(f"\nData summary:\n{df.describe()}")
    return df


def preprocess_data(df):
    df_clean = df.copy()
    print_section("📋 COLUMN ANALYSIS")

    print("\nColumn names:")
    for i, col in enumerate(df_clean.columns, 1):
        print(f"  {i}. '{col}' ({df_clean[col].dtype})")

    print(f"\n🔍 Missing values:\n{df_clean.isnull().sum()}")
    df_clean.columns = df_clean.columns.str.strip()

    print(f"\n✅ Data preprocessing complete. Shape: {df_clean.shape}")
    return df_clean

# ==================================================================================================
# 4. ANALYSIS (Adapted from analysis.py)
# ==================================================================================================

def exploratory_data_analysis(df_clean):
    print_section("📊 EXPLORATORY DATA ANALYSIS")

    print("\n🏎️ Sales by Model (Top 10):")
    print(df_clean.groupby('Model')['Sales_Volume'].sum().sort_values(ascending=False).head(10))

    print("\n🌍 Sales by Region:")
    print(df_clean.groupby('Region')['Sales_Volume'].sum().sort_values(ascending=False))

    print("\n📅 Sales by Year:")
    print(df_clean.groupby('Year')['Sales_Volume'].sum().sort_values())


def aggregate_time_series(df_clean):
    print_section("📈 TIME SERIES AGGREGATION")

    df_yearly = df_clean.groupby('Year')['Sales_Volume'].sum().reset_index()
    df_yearly = df_yearly.sort_values('Year')
    df_yearly.columns = ['Year', 'Total_Sales']

    print(f"\n✅ Yearly Sales Aggregation:\n{df_yearly}")

    ts_data = df_yearly['Total_Sales'].values
    ts_years = df_yearly['Year'].values

    df_yearly['YoY_Growth'] = df_yearly['Total_Sales'].pct_change() * 100

    df_model_yearly = df_clean.groupby(['Year', 'Model'])['Sales_Volume'].sum().reset_index()
    df_region_yearly = df_clean.groupby(['Year', 'Region'])['Sales_Volume'].sum().reset_index()

    return df_yearly, ts_data, ts_years, df_model_yearly, df_region_yearly

# ==================================================================================================
# 5. VISUALIZATION (Adapted from visualization.py)
# ==================================================================================================

def create_overview_visualizations(df_yearly, df_clean):
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    fig.suptitle('BMW Sales Overview (2010-2024)', fontsize=16, fontweight='bold')

    # 1. Overall Sales Trend
    ax1 = axes[0, 0]
    ax1.plot(df_yearly['Year'], df_yearly['Total_Sales'], marker='o', linewidth=2.5, markersize=8, color='#1f77b4', label='Total Sales')
    ax1.set_title('Total Sales Trend')
    ax1.grid(True, alpha=0.3)

    # 2. Year-over-Year Growth Rate
    ax2 = axes[0, 1]
    colors = ['green' if x > 0 else 'red' for x in df_yearly['YoY_Growth'].fillna(0)]
    ax2.bar(df_yearly['Year'][1:], df_yearly['YoY_Growth'][1:], color=colors[1:], alpha=0.7)
    ax2.set_title('Year-over-Year Growth Rate')
    ax2.axhline(y=0, color='black', linewidth=0.8)

    # 3. Sales by Model (Top 10)
    ax3 = axes[1, 0]
    model_total = df_clean.groupby('Model')['Sales_Volume'].sum().sort_values(ascending=True).tail(10)
    model_total.plot(kind='barh', ax=ax3, color='#ff7f0e', alpha=0.8)
    ax3.set_title('Top 10 Models by Sales')

    # 4. Sales by Region
    ax4 = axes[1, 1]
    region_total = df_clean.groupby('Region')['Sales_Volume'].sum().sort_values(ascending=False)
    colors_region = plt.cm.Set3(np.linspace(0, 1, len(region_total)))
    ax4.pie(region_total, labels=region_total.index, autopct='%1.1f%%', colors=colors_region, startangle=90)
    ax4.set_title('Sales Distribution by Region')

    plt.tight_layout()
    p = out_path('01_sales_overview.png')
    plt.savefig(p, dpi=300, bbox_inches='tight')
    print(f"✅ Saved: {p}")
    plt.close()


def create_heatmap(df_clean):
    heatmap_data = df_clean.pivot_table(values='Sales_Volume', index='Model', columns='Region', aggfunc='sum', fill_value=0)
    heatmap_data = heatmap_data.loc[heatmap_data.sum(axis=1).nlargest(15).index]

    plt.figure(figsize=(12, 10))
    sns.heatmap(heatmap_data, annot=True, fmt='.0f', cmap='YlOrRd')
    plt.title('Sales Heatmap: Model vs Region (Top 15 Models)')
    plt.tight_layout()
    p = out_path('02_model_region_heatmap.png')
    plt.savefig(p, dpi=300, bbox_inches='tight')
    print(f"✅ Saved: {p}")
    plt.close()


def _ci_to_bounds(ci_obj):
    """Normalize confidence-interval-like object to (lower, upper) 1D arrays."""
    if ci_obj is None:
        return None, None
    try:
        if hasattr(ci_obj, 'iloc'):
            lower = np.asarray(ci_obj.iloc[:, 0]).ravel()
            upper = np.asarray(ci_obj.iloc[:, 1]).ravel()
            return lower, upper
        arr = np.asarray(ci_obj)
        if arr.ndim == 2 and arr.shape[1] >= 2:
            return arr[:, 0].ravel(), arr[:, 1].ravel()
    except Exception:
        pass
    return None, None


def visualize_forecast(ts_data, ts_years, train_size, forecast_test_values, forecast_test_ci, future_values, future_years, future_ci):
    fig, ax = plt.subplots(figsize=(14, 6))
    ax.plot(ts_years, ts_data, marker='o', label='Historical Sales', color='#1f77b4')

    test_years = ts_years[train_size:]
    if forecast_test_values is not None:
        try:
            ax.plot(test_years, forecast_test_values, marker='s', linestyle='--', label='Test Forecast', color='#ff7f0e')
        except Exception:
            pass
        lower, upper = _ci_to_bounds(forecast_test_ci)
        if lower is not None and upper is not None and len(lower) == len(test_years):
            ax.fill_between(test_years, lower, upper, alpha=0.2, color='#ff7f0e')

    if future_values is not None and len(future_values) > 0:
        try:
            ax.plot(future_years, future_values, marker='^', linestyle=':', label='Future Forecast', color='#2ca02c')
        except Exception:
            pass
        lower_f, upper_f = _ci_to_bounds(future_ci)
        if lower_f is not None and upper_f is not None and len(lower_f) == len(future_years):
            ax.fill_between(future_years, lower_f, upper_f, alpha=0.2, color='#2ca02c')

    # safe guard for vertical line index
    split_idx = max(1, min(train_size - 1, len(ts_years) - 1))
    ax.axvline(x=ts_years[split_idx], color='red', linestyle='--', alpha=0.5, label='Train/Test Split')
    ax.set_title('BMW Total Sales: Historical Data & ARIMA Forecast')
    ax.legend()
    p = out_path('03_arima_forecast.png')
    plt.savefig(p, dpi=300, bbox_inches='tight')
    print(f"✅ Saved: {p}")
    plt.close()


def forecast_model_specific(df_model_yearly, top_models, model_thresholds):
    print_section("🏎️ MODEL-SPECIFIC FORECASTS")
    model_forecasts = {}
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    axes = axes.flatten()

    for idx, model in enumerate(top_models):
        model_data = df_model_yearly[df_model_yearly['Model'] == model].sort_values('Year')
        if len(model_data) > 2:
            try:
                sales = model_data['Sales_Volume'].values
                years = model_data['Year'].values
                model_arima = ARIMA(sales, order=(1, 1, 1)).fit()
                forecast_values = model_arima.get_forecast(steps=3).predicted_mean

                model_forecasts[model] = {
                    'historical': sales,
                    'forecast': forecast_values,
                    'years': years,
                    'forecast_years': np.array([years[-1] + i for i in range(1, 4)])
                }

                ax = axes[idx]
                ax.plot(years, sales, marker='o', label='Historical')
                ax.plot(np.array([years[-1] + i for i in range(1, 4)]), forecast_values, marker='^', linestyle='--', color='red', label='Forecast')
                ax.set_title(f'Model: {model}')
                ax.legend()
            except Exception as e:
                print(f"   ⚠️ Could not forecast {model}: {e}")

    plt.tight_layout()
    plt.savefig(out_path('04_model_forecasts.png'))
    plt.close()
    return model_forecasts


def create_interactive_dashboard(ts_years, ts_data, future_years, future_values, df_yearly, df_clean):
    print_section("📊 CREATING INTERACTIVE DASHBOARD")
    # allow pie charts in (2,2) with domain type
    fig = make_subplots(rows=2, cols=2,
                        specs=[[{'type':'xy'}, {'type':'xy'}], [{'type':'xy'}, {'type':'domain'}]],
                        subplot_titles=('Total Sales Trend', 'YoY Growth', 'Model Performance', 'Regional Distribution'))

    fig.add_trace(go.Scatter(x=ts_years, y=ts_data, name='Historical'), row=1, col=1)
    fig.add_trace(go.Scatter(x=future_years, y=future_values, name='Forecast', line=dict(dash='dash')), row=1, col=1)
    fig.add_trace(go.Bar(x=df_yearly['Year'][1:], y=df_yearly['YoY_Growth'][1:], name='Growth'), row=1, col=2)

    top_models = df_clean.groupby('Model')['Sales_Volume'].sum().nlargest(5).sort_values()
    fig.add_trace(go.Bar(y=top_models.index, x=top_models.values, orientation='h', name='Models'), row=2, col=1)

    region_dist = df_clean.groupby('Region')['Sales_Volume'].sum()
    fig.add_trace(go.Pie(labels=region_dist.index, values=region_dist.values, name='Regions'), row=2, col=2)

    fig.update_layout(height=900, width=1200, title_text="BMW Sales Analytics Dashboard")
    fig.write_html(out_path('05_interactive_dashboard.html'))
    print(f"✅ Saved: {out_path('05_interactive_dashboard.html')}")


def create_heatmap_interactive(df_model_yearly):
    pivot = df_model_yearly.pivot_table(values='Sales_Volume', index='Model', columns='Year', fill_value=0)
    pivot = pivot.loc[pivot.sum(axis=1).nlargest(10).index]

    fig = go.Figure(data=go.Heatmap(z=pivot.values, x=pivot.columns, y=pivot.index, colorscale='YlOrRd'))
    fig.update_layout(title='BMW Model Sales Trends (Top 10)', height=600)
    fig.write_html(out_path('06_model_heatmap_interactive.html'))
    print(f"✅ Saved: {out_path('06_model_heatmap_interactive.html')}")

# ==================================================================================================
# 6. FORECASTING (Adapted from forecasting.py)
# ==================================================================================================

def forecast_with_arima(ts_data, ts_years):
    print_section("🤖 ARIMA TIME SERIES FORECASTING")

    # ensure we have sensible train/test sizes
    n = len(ts_data)
    train_size = max(1, min(n-1, int(n * TRAIN_TEST_SPLIT)))
    train_data = ts_data[:train_size]
    test_data = ts_data[train_size:]

    try:
        model = ARIMA(train_data, order=ARIMA_ORDER).fit()
        forecast_test = model.get_forecast(steps=len(test_data))
        forecast_test_values = np.asarray(forecast_test.predicted_mean) if len(test_data)>0 else None
        forecast_test_ci = forecast_test.conf_int() if len(test_data)>0 else None

        full_model = ARIMA(ts_data, order=ARIMA_ORDER).fit()
        future_forecast = full_model.get_forecast(steps=FORECAST_STEPS)
        future_values = np.asarray(future_forecast.predicted_mean)
        future_ci = future_forecast.conf_int()

        future_years = np.array([ts_years[-1] + i for i in range(1, FORECAST_STEPS + 1)])
        print(f"\n🔮 FUTURE FORECAST (Next {FORECAST_STEPS} Years):")
        for y, v in zip(future_years, future_values):
            print(f"   Year {y:.0f}: {v:,.0f}")

    except Exception as e:
        print(f"⚠️ ARIMA error: {e}. Using Fallback.")
        # Fallback to Exponential Smoothing
        model = ExponentialSmoothing(ts_data, trend='add').fit()
        future_values = np.asarray(model.forecast(steps=FORECAST_STEPS))
        future_years = np.array([ts_years[-1] + i for i in range(1, FORECAST_STEPS + 1)])
        forecast_test_values = None
        forecast_test_ci = None
        future_ci = None

    return train_size, forecast_test_values, forecast_test_ci, future_values, future_years, future_ci

# ==================================================================================================
# 7. ALERTS SYSTEM (Adapted from alerts.py)
# ==================================================================================================

class SalesAlertSystem:
    def __init__(self, threshold, model_thresholds=None, region_thresholds=None):
        self.threshold = threshold
        self.model_thresholds = model_thresholds or {}
        self.region_thresholds = region_thresholds or {}
        self.alerts = []
        self.logger = setup_logger()

    def check_overall_forecast(self, forecast_values, threshold):
        alerts = []
        if forecast_values is None:
            return alerts
        for i, value in enumerate(forecast_values):
            try:
                if value < threshold:
                    alert = {
                        'type': 'OVERALL_SALES',
                        'severity': 'HIGH',
                        'message': f'ALERT: Forecast Year {i+1} ({value:,.0f}) < Threshold ({threshold:,.0f})',
                        'gap': float(threshold - value)
                    }
                    alerts.append(alert)
                    self.logger.warning(alert['message'])
            except Exception:
                continue
        return alerts

    def check_model_performance(self, model_data, model_name, threshold):
        alerts = []
        recent = None
        try:
            recent = model_data.get('historical')[-1] if model_data.get('historical') is not None else 0
        except Exception:
            recent = 0
        if recent < threshold:
            alert = {
                'type': 'MODEL_UNDERPERFORMANCE',
                'severity': 'MEDIUM',
                'model': model_name,
                'message': f'ALERT: Model {model_name} ({recent:,.0f}) < Threshold ({threshold:,.0f})',
                'gap': float(threshold - recent)
            }
            alerts.append(alert)
            self.logger.warning(alert['message'])
        return alerts

    def check_declining_trend(self, sales_history, item_name, decline_threshold=0.1):
        if sales_history is None or len(sales_history) < 2: return []
        try:
            decline_rate = (sales_history[-2] - sales_history[-1]) / max(1e-9, sales_history[-2])
        except Exception:
            return []
        if decline_rate > decline_threshold:
            alert = {
                'type': 'DECLINING_TREND', 'severity': 'MEDIUM',
                'message': f'ALERT: {item_name} showing {decline_rate*100:.1f}% decline'
            }
            self.logger.warning(alert['message'])
            return [alert]
        return []

    def generate_alert_report(self):
        print("\n" + "="*40 + "\nSALES ALERT REPORT\n" + "="*40)
        if not self.alerts: print("✅ No alerts triggered."); return
        for alert in self.alerts: print(f"   - [{alert.get('severity', 'INFO')}] {alert['message']}")

def setup_alert_system(df_clean, df_yearly, top_models):
    print_section("⚠️ ALERT THRESHOLD CONFIGURATION")
    avg_sales = df_yearly['Total_Sales'].mean()
    overall_thresh = avg_sales * OVERALL_THRESHOLD_MULTIPLIER

    model_thresholds = {m: df_clean[df_clean['Model']==m]['Sales_Volume'].mean()*MODEL_THRESHOLD_MULTIPLIER for m in top_models}
    region_thresholds = {r: df_clean[df_clean['Region']==r]['Sales_Volume'].mean()*REGION_THRESHOLD_MULTIPLIER for r in df_clean['Region'].unique()}

    return SalesAlertSystem(overall_thresh, model_thresholds, region_thresholds), overall_thresh, model_thresholds, region_thresholds, df_clean['Region'].unique()

# ==================================================================================================
# 8. REPORTING & AGGREGATOR (Adapted from reporting.py & aggregator.py)
# ==================================================================================================

def generate_monthly_report(alerts, future_values, ts_data, future_years, threshold):
    report = f"\nBMW SALES REPORT - {datetime.now().strftime('%Y-%m-%d')}\n"
    report += f"Total Forecast (Next 3 Years Avg): {np.mean(future_values):,.0f}\n" if future_values is not None else ''
    report += f"Active Alerts: {len(alerts)}\n"
    report += f"Alert Threshold: {threshold:,.0f}\n"
    return report

def export_data(future_years, future_values, threshold, alert_system, model_forecasts, model_thresholds):
    print_section("📊 EXPORTING DATA")
    if future_values is not None:
        pd.DataFrame({
            'Year': future_years.astype(int),
            'Forecast': future_values,
            'Threshold': threshold
        }).to_csv(out_path('forecast_next_3_years.csv'), index=False)
    if getattr(alert_system, 'alerts', None):
        pd.DataFrame(alert_system.alerts).to_csv(out_path('active_alerts.csv'), index=False)
    print("✅ CSV files exported.")

def create_aggregator_html():
    out_html = out_path('07_all_outputs.html')
    pngs = sorted([os.path.basename(p) for p in glob.glob(str(OUTPUT_DIR / '*.png'))])
    htmls = sorted([os.path.basename(p) for p in glob.glob(str(OUTPUT_DIR / '*.html')) if '07' not in p])
    content = "<html><body><h1>BMW Sales Forecast Outputs</h1>"
    if pngs: content += "<h2>Visualizations</h2>" + "".join([f"<figure><figcaption>{p}</figcaption><img src='{p}' width='600'/></figure>" for p in pngs])
    if htmls: content += "<h2>Interactive</h2>" + "".join([f"<div><a href='{h}' target='_blank'>Open {h}</a></div>" for h in htmls])
    content += "</body></html>"
    with open(out_html, 'w') as f: f.write(content)
    print(f"✅ Aggregator created: {out_html}")
    try: webbrowser.open(Path(out_html).as_uri())
    except: pass

# ==================================================================================================
# 9. MAIN EXECUTION
# ==================================================================================================

def run_alert_checks(alert_system, future_values, model_forecasts, df_region_yearly,
                     overall_thresh, model_thresh, region_thresh, regions, latest_year):

    alert_system.alerts.extend(alert_system.check_overall_forecast(future_values, overall_thresh))

    for model, data in (model_forecasts or {}).items():
        alert_system.alerts.extend(alert_system.check_model_performance(data, model, model_thresh.get(model, overall_thresh)))
        alert_system.alerts.extend(alert_system.check_declining_trend(data.get('historical'), model, DECLINE_THRESHOLD))

    for region in regions:
        latest = df_region_yearly[(df_region_yearly['Region']==region) & (df_region_yearly['Year']==latest_year)]['Sales_Volume'].values
        if len(latest) > 0 and latest[0] < region_thresh.get(region, overall_thresh):
            alert_system.alerts.append({'type': 'REGION', 'message': f'Region {region} underperforming'})

    return alert_system


def main():
    print_section("🚀 BMW SALES FORECASTING SYSTEM INITIALIZED")

    # 1. Data
    download_required_files()
    df = load_and_explore_data(DATA_CSV_FILE)
    df_clean = preprocess_data(df)
    exploratory_data_analysis(df_clean)

    # 2. Aggregation & Viz
    df_yearly, ts_data, ts_years, df_model_yearly, df_region_yearly = aggregate_time_series(df_clean)
    create_overview_visualizations(df_yearly, df_clean)
    create_heatmap(df_clean)

    # 3. Forecasting
    train_size, forecast_test, forecast_ci, future_values, future_years, future_ci = forecast_with_arima(ts_data, ts_years)
    visualize_forecast(ts_data, ts_years, train_size, forecast_test, forecast_ci, future_values, future_years, future_ci)

    top_models = df_clean.groupby('Model')['Sales_Volume'].sum().nlargest(5).index.tolist()
    model_forecasts = forecast_model_specific(df_model_yearly, top_models, {})

    # 4. Alerts
    alert_sys, overall_thresh, model_thresh, region_thresh, regions = setup_alert_system(df_clean, df_yearly, top_models)
    latest_year = df_region_yearly['Year'].max()
    alert_sys = run_alert_checks(alert_sys, future_values, model_forecasts, df_region_yearly, overall_thresh, model_thresh, region_thresh, regions, latest_year)
    alert_sys.generate_alert_report()

    # 5. Reporting
    print(generate_monthly_report(alert_sys.alerts, future_values, ts_data, future_years, overall_thresh))
    create_interactive_dashboard(ts_years, ts_data, future_years, future_values, df_yearly, df_clean)
    create_heatmap_interactive(df_model_yearly)
    export_data(future_years, future_values, overall_thresh, alert_sys, model_forecasts, model_thresh)
    create_aggregator_html()

    print("\n✅ PROCESS COMPLETE.")

if __name__ == "__main__":
    main()
